In [132]:
import time
import subprocess
from os import listdir
from os.path import isfile, join
import sklearn.datasets
import nltk.stem
import numpy as np
import pandas as pd
from stop_words import get_stop_words
import re
import scipy as sp
import PyPDF2
import docx
import sys
import os
import os.path
import string
import docx
import docx2txt
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer

In [133]:
stop_words = get_stop_words('english')
english_stemmer = nltk.stem.SnowballStemmer('english')

In [134]:
def file_operation():
    data = []
    mypath = "Users/sahebsingh/Desktop/Projects/Honeywell/Honeywell" #Path to folder/Directory
    
    list_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    for k in range(len(list_files)):
        document = docx.Document(list_files[k])
    print (document.paragraphs)

In [135]:
def getPdf(file):
    data = []
    file = open(file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(file)
    page_number = pdfReader.numPages
    for i in range(0, page_number):
        pageObj = pdfReader.getPage(i)
        data.append(pageObj.extractText())
        
    return '\n'.join(data)
    

In [136]:
def getDocx(filename):
    my_text = docx2txt.process(filename)
    
    return np.array(my_text)
    

In [137]:
data1 = getDocx('2.docx')

In [138]:
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)
    
    

In [139]:
def getText1(filename):
    from docx import Document
    document = Document(filename)
    for para in document.paragraphs:
        print(para.text)

In [140]:
data = getPdf('2.pdf')

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
articleURL = "http://curia.europa.eu/juris/document/document.jsf?text=&docid=139407&pageIndex=0&doclang=EN&mode=lst&dir=&occ=first&part=1&cid=52454"
def getText(url):
    page = urlopen(url).read().decode('utf8', 'ignore')
    soup = BeautifulSoup(page, 'lxml')
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
    return text.encode('ascii', errors='replace').decode().replace("?","")
text = getText(articleURL)

In [ ]:
text

In [144]:
import nltk
# nltk.download('punkt')
# nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from string import punctuation
from heapq import nlargest
def summarize(text, n):
    sents = sent_tokenize(text)
    
    assert n <= len(sents)
    wordSent = word_tokenize(text.lower())
    stopWords = set(stopwords.words('english')+list(punctuation))
    
    wordSent= [word for word in wordSent if word not in stopWords]
    freq = FreqDist(wordSent)
    ranking = defaultdict(int)
    
    for i, sent in enumerate(sents):
        for w in word_tokenize(sent.lower()):
            if w in freq:
                ranking[i] += freq[w]
    sentsIDX = nlargest(n, ranking, key=ranking.get)
    return [sents[j] for j in sorted(sentsIDX)]
summaryArr = summarize(data, 10)
# summaryArr

In [195]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
t = 4
vectorizer = TfidfVectorizer(max_df=0.5,min_df=2,stop_words='english')
X = vectorizer.fit_transform(summaryArr)
km = KMeans(n_clusters = t, init = 'k-means++', max_iter = 100, n_init = 1, verbose = True)
km.fit(X)
np.unique(km.labels_, return_counts=True)
text={}
for i,cluster in enumerate(km.labels_):
    oneDocument = summaryArr[i]
    if cluster not in text.keys():
        text[cluster] = oneDocument
    else:
        text[cluster] += oneDocument
stopWords = set(stopwords.words('english')+list(punctuation))
keywords = {}
counts={}
for cluster in range(t):
    word_sent = word_tokenize(text[cluster].lower())
    word_sent=[word for word in word_sent if word not in stopWords]
    freq = FreqDist(word_sent)
    keywords[cluster] = nlargest(100, freq, key=freq.get)
    counts[cluster]=freq
uniqueKeys={}
for cluster in range(t):   
    other_clusters=list(set(range(t))-set([cluster]))
    keys_other_clusters=set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
    unique=set(keywords[cluster])-keys_other_clusters
    uniqueKeys[cluster]=nlargest(10, unique, key=counts[cluster].get)
print(uniqueKeys)

Initialization complete
Iteration  0, inertia 8.299
Iteration  1, inertia 4.837
Converged at iteration 1: center shift 0.000000e+00 within tolerance 7.394008e-07
{0: ['rsm', 'sr.', 'national', 'east', 'business', 'midwest', 'interiors', 'region', 'central', 'west'], 1: ['aircraft', 'update', 'bulletin', 'information', 'technical', 'trim', '...', 'instrument', 'maintenance', 'cb'], 2: ['communications', 'corporate', 'fedak', 'heidi', 'promoted', '14all', 'baugniet', 'highlights', 'reporting', 'robert'], 3: ['updates', 'loading', 'charlie', 'certification', 'dls', 'modules', 'orientation', 'https', 'data', 'check']}


In [166]:
post = """A focus on safety
"""

In [167]:
new_post_vec = vectorizer.transform([post])
new_post_label = km.predict(new_post_vec)[0]#We will predict it's cluster 
# Comparing all 
similar_indices = (km.labels_ == new_post_label).nonzero()[0]


In [168]:
similar = []
for i in similar_indices:
    dist = sp.linalg.norm((new_post_vec - X[i]).toarray())
    similar.append((dist, summaryArr[i]))


In [169]:
similar = sorted(similar) 
print(len(similar))
show_at_1 = similar[0]
show_at_2 = similar[2]
show_at_3 = similar[3]

5


In [170]:
print("=== #1 ===")
print(show_at_1)
print()


=== #1 ===
(1.2763020409819394, '1   Volume 16, Issue 19 November 14, 2008 Volume 16, Issue 19      \n \n \n \n   \n \n \n \n \n   \n \n \n \n  A FOCUS ON SAFETY ALL:\n The Gulfstream manuals now call only for bonding \nthe fuel source to the aircraft during fueling or defueling...2                                   TOP STORIES ALL: \nAn overview of the large-cabin AMM revisions \nTechnical Publications has produced to date in 2008–\n3  \nAstraŽ/SPXŽ/G100\n®/G150®/G200®: Rudder Trim \ninstallations on the Astra through G200 aircraft are \nsusceptible to water ingress into the trim actuator \nhousings...4  G150®: ﬁGEN LOAD UNBALANCEﬂ status messages \nhave been traced to a poor gr\nound for the APU generator \nreturn cable at 10GND...5  G200®: An issue surrounding the operational \ncharacteristics of the Entr\ny Light Switch Panel (CMS1 \nCabin System) has been brought \nto our attention by a \nG200 operator...5  GII®/GIII®: During left engine start, when the start button \nwas depresse

In [171]:
print("=== #2 ===")
print(show_at_2)
print()



=== #2 ===
(1.4142135623730951, 'BREAKFAST MINUTES                              November 14, 2008    13 Ł \nBreakfast Minutes Tips \nŠ Here are some useful tips to help you get the most out of \nBreakfast Minutes resources:  Œ Printing the entire \nBreakfast Minutes issue Š\n From myGulfstream.com, navigate to the \nBreakfast Minutes home page (click Resources Œ> Breakfas\nt Minutes), select the PDF Version for \nthe particular issue you want to print (Adobe\n® Acrobat\n® Reader is required).')



In [172]:
print("=== #3 ===")
print(show_at_3)



=== #3 ===
(1.4142135623730951, 'MAINTENANCE SALES TEAM  16  Regional Maintenance Sales Team \nUpcoming Events November 17 Œ Forum, Reception, Dubai, UAE \nDecember 3 Œ Forum, CMP.net, Glendale, Calif. \nJune 8-11, 2009ŒGulfstream O\nperators™ Conference\nBREAKFAST MINUTES                              November 14, 2008    3TOP STORIES  Large-Cabin (ATA 00): Aircraft Maintenance Manual Update By Skip Weinrick, Sr.')
